In [1]:
import numpy as np
import scipy
import scipy.sparse as sps
import porepy as pp

import ddf.common as co
import ddf.hrl as hrl
from ddf.dks import DKS

from ddf.plot import quiver, plot_cells, streamplot
from ddf.immagini import *

from tabulate import tabulate
import pickle
import gc

In [2]:
pbs = {}
gbs = {}

eigs = {}
gigs = {}

def run(nome, vg15_kwargs, eig_kwargs={}, dati_kwargs={}):
    print(nome, end=' ')

    imp = dict(cartella='../simulazioni/eig_vg15', nome=nome, parla=4, campi_post=[])
    pb = co.pkl(imp)

    vg15_kwargs = dict(tipo_griglia='quadrata') | vg15_kwargs
    eig_kwargs = dict(post=1, k=8) | eig_kwargs
    
    if pb is None:
        print('s')
        mdg, dati = hrl.vg15(**vg15_kwargs)
        dati = dati | dati_kwargs
        pb = hrl.HRL(mdg, dati, imp)
        pb.init()
        pb.steady_state()
        pb.esporta(stato=1, pkl=1)
    
    if not hasattr(pb, '_eig'):
        print('e')
        eig = pb.eig(**eig_kwargs)
        print(f'({eig.matvec_count}) ', eig.vals)
        pb.esporta(stato=1, pkl=1)

    return pb

In [3]:
# figura = '9a'; lettera = 'C_'; nome = f'{figura}{lettera}'
# pb = run(nome, dict(figura=figura, lettera=lettera, grid_scale=2.50), dict(ks=1, tol=1e-15, k=8));

In [4]:
# g=1.75; 2/(10/20/g)
# 1/(g**3/8)

# for lettera in ['D']:
for lettera in ['A', 'B', 'C']:
    nome = f'5{lettera}'; gome = nome + 'g'
    gbs[nome] = run(gome, dict(figura='5', lettera=lettera, grid_scale=1.75), dict(ks=1, tol=1e-15));
    pbs[nome] = run(nome, dict(figura='5', lettera=lettera, grid_scale=3.00), dict(ks=1, tol=1e-15));
    eigs[nome] = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome=nome), eig=1)
    gigs[nome] = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome=gome), eig=1)

5Ag s
0.00: (0) sol: [   0.312    0.986        0        0] theta: 0.0
0.00: (1) sol: [   0.312    0.986  3.2e-08 2.46e-06]  inc: [6.38e-05 1.29e-06  3.2e-08 2.46e-06]
0.00: (2) sol: [   0.312    0.986  3.2e-08 2.46e-06]  inc: [5.36e-09 1.46e-09 2.13e-10  4.4e-11]

e


KeyboardInterrupt: 

In [ ]:
for lettera in ['A', 'B', 'B^', 'B^^', 'C']:
    nome = f'6{lettera}'; gome = nome + 'g'
    pb = run(nome, dict(figura='6', lettera=lettera, grid_scale=2.00), dict(ks=1, tol=1e-15, k=14));
    gb = run(gome, dict(figura='6', lettera=lettera, grid_scale=1.75), dict(ks=1, tol=1e-15, k=8));
    eigs[nome] = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome=nome), eig=1)
    gigs[nome] = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome=gome), eig=1)


In [ ]:
for figura in ['9a', '9b']:
    for lettera in ['A', 'B', 'C_', 'C', 'D', 'E', 'F']:
        if figura == '9b' and lettera == 'C_': continue
        nome = f'{figura}{lettera}'
        pbs[nome] = run(nome, dict(figura=figura, lettera=lettera, grid_scale=2.50), dict(ks=1, tol=1e-15));
        eigs[nome] = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome=nome), eig=1)

In [ ]:
# tabella = [
#     { nome: nome } |
#     { f'lambda_{j}': eigs[nome].vals[j] for j,val in enumerate(} for 

k = 8

nomi = [ nome for nome in eigs.keys() ]
# nomi = [ nome for nome in eigs.keys() if nome in gigs.keys() ]
def err_rel_griglia(nome):
    if not nome in gigs.keys(): return [0]
    
    vals = eigs[nome].vals
    gals = gigs[nome].vals
    err_rel = 1 - gals[:k]/vals[:k]
    return np.abs(err_rel)

tabella = [
    { 'nome': nome } | 
#     { f'l{i}': val for i,val in enumerate(pbs[nome]._eig.vals) }
    { f'l{j}': eigs[nome].vals[j] for j in range(k) } |
#     { f'eg{j}': err_rel_griglia(nome)[j] for j in range(k) } |
    {
        'eps': np.linalg.norm([eigs[nome].err_rel[i] for i in range(5)], np.inf),
#         'eps_g': np.linalg.norm(err_rel_griglia(nome), np.inf),
#         'dof/dofg': eigs[nome].funs.shape[0] / gigs[nome].funs.shape[0],
        'dof': int(eigs[nome].funs.shape[0]), 
        'mv': eigs[nome].matvec_count,
    }
    for nome in nomi
]
 
print(tabulate(tabella, floatfmt='.2f', headers='keys'))

In [ ]:
for nome,eig in eigs.items():
    print(f'{nome:<5}' + ' '.join([ f'{v:>8.2f}' for v in eig.vals ]))
    
print('')
for nome,eig in gigs.items():
    print(f'{nome:<5}' + ' '.join([ f'{v:>8.2f}' for v in eig.vals ]))

In [ ]:
def peigs(eig, axs):
    for i,ax in enumerate(axs.flatten()):
        img = Image.open(f'../immagini/paraview/{nome}_{i}.png')
        ax.imshow(img, extent=[0, 1, 0, 1])
        val = eig.vals[i]
        bbox = dict(facecolor='white', boxstyle='square,pad=0.3', edgecolor='black' if val > 0 else 'lightgrey')
        ax.text(1, 0, r'$\lambda_{%d}$ = %.2f' % (i+1, val), fontsize='small', va='bottom', ha='right', bbox=bbox)

In [ ]:
nome = f'5C'
fig,axs = plt.subplots(2,4, figsize=(linewidth, linewidth/4*2*1.03), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs)
fig.tight_layout(pad=0.6)
fig.savefig(f'../immagini/vg15_{nome}.png', transparent=1, bbox_inches='tight')

In [ ]:
nome = '6B^'
fig,axs = plt.subplots(2,2, figsize=(linewidth*.5, linewidth*.5/2*2*.99), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs.flatten())

for mm in ['bottom', 'top', 'right', 'left']: 
    axs[1,0].spines[mm].set_color('#ff6d6d')
    axs[1,0].spines[mm].set_linewidth(2.3)

for mm in ['bottom', 'top', 'right', 'left']:
    for ax in [ axs[0,0], axs[0,1] ]:
        ax.spines[mm].set_color('#729fcf')
        ax.spines[mm].set_linewidth(2.3)

    
fig.tight_layout(pad=0.45)
fig.savefig(f'../immagini/vg15_{nome}.png', **sf_kw|dict(pad_inches=0.05))

In [ ]:
nome = '6C'
fig,axs = plt.subplots(3,5, figsize=(linewidth, linewidth/5*3*1.03), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs.flatten()[:14])
axs.flatten()[14].set_axis_off()
fig.tight_layout(pad=0.6)
fig.savefig(f'../immagini/vg15_{nome}.png', transparent=1, bbox_inches='tight')

In [ ]:
nome = '9aD'
fig,axs = plt.subplots(2,2, figsize=(linewidth*.5, linewidth*.5/2*2*.99), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs.flatten())

for mm in ['bottom', 'top', 'right', 'left']: 
    axs[1,0].spines[mm].set_color('#ff6d6d')
    axs[1,0].spines[mm].set_linewidth(2.3)

for mm in ['bottom', 'top', 'right', 'left']:
    for ax in [ axs[0,0], axs[0,1] ]:
        ax.spines[mm].set_color('#729fcf')
        ax.spines[mm].set_linewidth(2.3)


fig.tight_layout(pad=0.4)
# fig.savefig(f'../immagini/vg15_{nome}.png', **sf_kw|dict(pad_inches=0.05))

In [ ]:
nome = '9bF'
fig,axs = plt.subplots(2,4, figsize=(linewidth, linewidth/4*2*1.03), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs)
fig.tight_layout(pad=0.6)
fig.savefig(f'../immagini/vg15_{nome}.png', transparent=1, bbox_inches='tight')

In [ ]:
# %run /home/arash/src/ddf/paraview/meig.py

In [ ]:
# nome = '6B^'
nome = '9aC'
fig,axs = plt.subplots(2,4, figsize=(linewidth, linewidth/4*2*1.03), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs)
fig.tight_layout(pad=0.6)
# fig.savefig(f'../immagini/vg15_{nome}.png', transparent=1, bbox_inches='tight')

In [ ]:
nome = '6B^^'
fig,axs = plt.subplots(2,4, figsize=(linewidth, linewidth/4*2*1.03), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs[nome], axs)
fig.tight_layout(pad=0.6)
fig.savefig(f'../immagini/vg15_{nome}.png', transparent=1, bbox_inches='tight')

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(linewidth/2, linewidth/2/2), subplot_kw=dict(xticks=[], yticks=[]))
e = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome='5A'), eig=1); peigs(e, np.array([ axs[0] ]))
e = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome='5B'), eig=1); peigs(e, np.array([ axs[1] ]))
fig.tight_layout(pad=0.3)
fig.savefig('../immagini/vg15_5.png', **sf_kw|dict(pad_inches=0.05))

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(linewidth/2, linewidth/2/2), subplot_kw=dict(xticks=[], yticks=[]))
e = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome='6A'), eig=1); peigs(e, np.array([ axs[0] ]))
e = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome='6B'), eig=1); peigs(e, np.array([ axs[1] ]))
fig.tight_layout(pad=0.3)
fig.savefig('../immagini/vg15_6.png', **sf_kw|dict(pad_inches=0.05))

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(linewidth/2, linewidth/2/2), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs['9aB'], np.array([ axs[0] ]))
peigs(eigs['9aC'], np.array([ axs[1] ]))
fig.tight_layout(pad=0.3)
fig.savefig('../immagini/vg15_9a.png', **sf_kw)

In [ ]:
fig,axs = plt.subplots(1,2, figsize=(linewidth/2, linewidth/2/2), subplot_kw=dict(xticks=[], yticks=[]))
peigs(eigs['9bB'], np.array([ axs[0] ]))
peigs(eigs['9bC'], np.array([ axs[1] ]))
fig.tight_layout(pad=0.3)
fig.savefig('../immagini/vg15_9b.png', **sf_kw)

In [ ]:
raise

In [ ]:
pb = run('quadrata_semplice', dict(figura='5', lettera='D', grid_scale=2.0, tipo_griglia='quadrata'), dict(ks=1, tol=1e-15), dict(newton_atol=1e-12));
# pb = run('triangolare_semplice', dict(figura='5', lettera='D', grid_scale=1.0, tipo_griglia='triangolare'), dict(ks=1, tol=1e-15), dict(newton_atol=1e-13));
# pb = run('triangolare_vicinanza', dict(figura='5', lettera='D', grid_scale=1.0, tipo_griglia='triangolare'), dict(ks=1, tol=1e-15), dict(newton_atol=1e-13));
# pb = run('triangolare_volumi', dict(figura='5', lettera='D', grid_scale=1.0, tipo_griglia='triangolare'), dict(ks=1, tol=1e-15), dict(newton_atol=1e-13));

In [ ]:
pb = run('triangolare_semplice', dict(figura='5', lettera='D', grid_scale=1.0, tipo_griglia='triangolare'), dict(ks=1, tol=1e-15), dict(newton_atol=1e-13));

In [ ]:
pb = run('triangolare_volumi', dict(figura='5', lettera='D', grid_scale=1.0, tipo_griglia='triangolare'), dict(ks=1, tol=1e-15), dict(newton_atol=1e-12));

In [ ]:
pb = co.pkl(dict(cartella='../simulazioni/eig_vg15', nome='6C'))
ks = DKS(pb.eig_ops().S, 8, 64, b=pb._M.diagonal(), tol=1e-15)
ks.itera(maxmv=10_000, log_freq=1, pkl_path=f'{pb.cartella_out}/eig_ks.gz')

_,funs = ks.eig(completi=1)
funs = funs[:,:ks.l]
assert np.allclose(np.imag(funs), 0)
funs = np.real(funs)
pb.eig_post(funs)

In [ ]:
qbs[nome]._ks.errori(completi=1)

In [ ]:
_,funs = ks.eig(completi=1)
funs = funs[:,:ks.l]
assert np.allclose(np.imag(funs), 0)
funs = np.real(funs)
pb.eig_post(funs)
pb.esporta(stato=1, pkl=1)

In [ ]:
vals,funs = ks.eig(completi=1)
pb.eig_post(funs)
pb.esporta(stato=1, pkl=1)

In [ ]:
nome = '5B'
vals = np.stack(( gbs[nome]._eig.vals, pbs[nome]._eig.vals, fbs[nome]._eig.vals ))
for i in range(3): plt.scatter( np.arange(8), vals[i] )